# Let's pull in our data -- writings from Black abolitionists 

### Doc set up

In [7]:
import os
import random
from bs4 import BeautifulSoup
import requests
from string import ascii_uppercase 

In [8]:
# We need to create base URLS for later in the process

base_url = 'https://libraries.udmercy.edu/find/special_collections/digital/baa/'
base_url_letter = 'https://libraries.udmercy.edu/find/special_collections/digital/baa/index.php?browseBy=DC_creator&letter={}&cloud=n'

In [9]:
# This is how the {} will work...here's an example:
#text = 'test{}'
#text.format('love')

### Function to sift through each letter for author URLS

In [20]:
def find_author_URL(letter_URL):
    """
    
    This function sifts through every letter URL -- 
    e.g. the webpage with all A authors -- 
    to pull out the document page URL.
    
    ARGUMENT: letter_URL is the URL for each letter webpage
    VALUES: URLS for each author's homepage listed on the initial link
    
    """
    
    locate_letter_page = requests.get(letter_URL)
    letter_page_text = BeautifulSoup(locate_letter_page.text, 'html.parser')
    author_codes = letter_page_text.find_all('p')

    author_links = []

    for author_code in author_codes:
        author_link = base_url + author_code.find('a').attrs['href'] #find all author links
        author_links.append(author_link)

    
    return author_links

### Function to sift through all document of a given author

In [26]:
def find_all_documents(author_URL):
    
    """
    
    This function helps sift through all the documents
    of a given author's page.(Most have one, but some have multiple.)
    At the end, it spits out each specific document page links.
    
    ARGUMENT: author_URL is the webpage on which all documents are listed that they wrote
    VALUES: URLS for each document written by the author of the URL
    
    """
    
    locate_author_documents_page = requests.get(author_URL)
    author_documents_page = BeautifulSoup(locate_author_documents_page.text, 'html.parser')

    all_links = author_documents_page.find_all('a', class_='listLink') #find each link's general location

    document_links = [] #create new list to hold all the document links
    for link in all_links:
        document_link = base_url + link['href'] #paste the base link together with the cleaned up found link
        document_links.append(document_link)
    
    return document_links

In [35]:
## TEST?

#url = "https://libraries.udmercy.edu/find/special_collections/digital/baa/index.php?collectionCode=baa&field=DC_creator&term=%22Wake%2C+Ransom+F.%22"
url = 'https://libraries.udmercy.edu/find/special_collections/digital/baa/index.php?collectionCode=baa&field=DC_creator&term=%22White%2C+Jacob+C.%2C+d.+1872%22'
documents_URL = find_all_documents(url)

### Function to pull out each PDF and download it

In [249]:
def final_PDF_URL(document_URL, output_folder):
    
    """
    This function provides the final main step of my crawling. 
    It will find the PDF links for URL's given document.
    
    ARGUMENT: document_URL is the page that houses the PDF links
    VALUES: Downlaod  each PDF document.
    """
    
    #LOADING IN THE page with the PDF
    locate_document_page = requests.get(document_URL)
    document_page = BeautifulSoup(locate_document_page.text, 'html.parser')

    document_page.text.find('Click to view PDF')

    if document_page.text.find('Click to view PDF') == -1:
        print('not a PDF')
        return None

    #Search for bolded (aka important) notes regarding the data
    key_terms_dict = {}
    keys = []
    values = []
    for p in document_page.find_all('p'):

        try: 
            if len(p.find('strong')):

                keys.append(p.text.split(": ")[0])
                values.append(p.text.split(": ")[1])

        except:
            print("No 'strong' in this line")

    for i in range(len(keys)):
        key_terms_dict[keys[i]] = values[i]

    # not all have date published...fill that in here
    try:
        if len(key_terms_dict['Date published'])>1:
            key_terms_dict['Data_note'] = "Complete"
    except:
        key_terms_dict['Date published'] = "Unknown"
        key_terms_dict['Data_note'] = "Incomplete"
    
    # Find the PDF Link 
    PDF_URL = document_page.find('td').find('a')['href']
    
    # NEED HELP!!
    # Download PDF LINK -- NEED HELP
    PDF = requests.get(PDF_URL)

    
    # name file by Title (author) and date published
    filename = f'{key_terms_dict["Title"]}_{key_terms_dict["Date published"]}'

    # in case a file already exists, add digits at the end [TAKEN FROM JAKE DIRECTLY]
    if os.path.isfile(os.path.join(output_folder,filename+'.pdf')):
        filename += "_" + str(random.randint(100,999))

    # Create the file name in the dictionary [TAKEN FROM JAKE DIRECTLY]
    key_terms_dict['filename'] = filename + '.pdf'

    # save PDF [TAKEN FROM JAKE DIRECTLY]
    print(f'saving -- {filename}.pdf')
    PDF = requests.get(PDF_URL)
    with open(os.path.join(output_folder,filename+'.pdf'),'wb') as pdf:
        pdf.write(PDF.content)
    return key_terms_dict

## scratch for the final function

In [219]:
#documents_URL[0]

'https://libraries.udmercy.edu/find/special_collections/digital/baa/item.php?record_id=2605&collectionCode=baa'

In [260]:
#document_URL= 'https://libraries.udmercy.edu/find/special_collections/digital/baa/item.php?record_id=2135&collectionCode=baa'
#locate_document_page = requests.get(document_URL)
#document_page = BeautifulSoup(locate_document_page.text, 'html.parser')

#key_terms_dict = {}
#keys = []
#values = []
#for p in document_page.find_all('p'):
    

#    try: 
#        if len(p.find('strong')):

#            keys.append(p.text.split(": ")[0])
#            values.append(p.text.split(": ")[1])
#               
#    except:
#        print("No 'strong' in this line")

#for i in range(len(keys)):
##    key_terms_dict[keys[i]] = values[i]
        
#key_terms_dict

No 'strong' in this line
No 'strong' in this line


{'Description of file(s)': 'PDF 3 page, 630 word document (text and images)',
 'Keywords': 'Mental Feast; moral development; Philadelphia; women',
 'Newspaper or publication': 'Liberator',
 'Publication type': 'Newspapers; Speeches',
 'Speaker or author': 'Douglass, S. M. (Sarah Mapps), 1806-1882',
 'Subjects': 'Abolitionists--United States; African American abolitionists; Antislavery movements--United States; Slavery; United States--History--19th century',
 'Title': 'Sarah M. Douglass'}

In [264]:
#try:
#    if len(key_terms_dict['Date published'])>1:
#        print("Has date published")
#except:
#    key_terms_dict['Date published'] = "Unknown"
#    print('new made')
#    print(key_terms_dict['Date published'])

Has date published


In [255]:
#LOADING IN THE page with the PDF

#document_URL = 'https://libraries.udmercy.edu/find/special_collections/digital/baa/item.php?record_id=2135&collectionCode=baa'
#document_URL= 'https://libraries.udmercy.edu/find/special_collections/digital/baa/item.php?record_id=963&collectionCode=baa'
#locate_document_page = requests.get(document_URL)
#document_page = BeautifulSoup(locate_document_page.text, 'html.parser')

#document_page.text.find('Click to view PDF')

#if document_page.text.find('Click to view PDF') == -1:
#    print('not a PDF')
#    #return none

#document_page.text
#pick out key info regarding the data_set

ConnectionError: HTTPSConnectionPool(host='libraries.udmercy.edu', port=443): Max retries exceeded with url: /find/special_collections/digital/baa/item.php?record_id=963&collectionCode=baa (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f167186a8d0>: Failed to establish a new connection: [Errno -2] Name or service not known'))

In [223]:
#Search for bolded (aka important) notes regarding the data
#key_terms_dict = {}
#keys = []
#values = []
#for p in document_page.find_all('p'):
    

#    try: #
#        if len(p.find('strong')):

 #           keys.append(p.text.split(": ")[0])
  #          values.append(p.text.split(": ")[1])
               
  #  except:
  #      print("No 'strong' in this line")

#for i in range(len(keys)):
#    key_terms_dict[keys[i]] = values[i]
        
#key_terms_dict

No 'strong' in this line
No 'strong' in this line


{'Date published': '1859-08',
 'Description of file(s)': 'PDF 4 page, 854 word document (text and images)',
 'Keywords': 'August 1st; British; celebration; emancipation; England; freedom; Jamaica; West Indies',
 'Newspaper or publication': 'Weekly Anglo-African (1859 - 1862)',
 'People': 'Bleby, Rev. Henry; Clarkson, Thomas; Wilberforce, William',
 'Publication type': 'Newspapers; Speeches',
 'Speaker or author': 'White, Jacob C., d. 1872',
 'Subjects': 'Abolitionists--United States; African American abolitionists; Antislavery movements--United States; Slavery; United States--History--19th century',
 'Title': 'Jacob C. White, Jr.'}

In [235]:
# NEED HELP!!
# Download PDF LINK -- NEED HELP
#PDF = requests.get(PDF_URL)

# name file by Title (author) and date published
#filename = f'{key_terms_dict["Title"]}_{key_terms_dict["Date published"]}'

# in case a file already exists, add digits at the end [TAKEN FROM JAKE DIRECTLY]
#if os.path.isfile(os.path.join('sam_test',filename+'.pdf')):
#    filename += "_" + str(random.randint(100,999))

# Find the PDF Link 
#PDF_URL = document_page.find('td').find('a')['href']

# Create the file name in the dictionary
#key_terms_dict['filename'] = filename + '.pdf'


# save PDF
#print(f'saving -- {filename}.pdf')
#PDF = requests.get(PDF_URL)
#with open(os.path.join('sam_test',filename+'.pdf'),'wb') as pdf:
#    pdf.write(PDF.content)
#return key_terms_dict

saving -- Jacob C. White, Jr._1859-08_587.pdf


### Now, let's bring the pieces together

In [238]:
#ascii_uppercase

'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [251]:
document_json = []

#sift through every letter page for the authors on it
for letter in ascii_uppercase:  #just do A for now
    letter_URL = base_url_letter.format(letter) #find the URL for the first letter, etc
    
    authors_URL = find_author_URL(letter_URL) #find the URL for each author on the letter page 

    # sift through the authors page for all of their documents
    for author_URL in authors_URL:
        documents_URL = find_all_documents(author_URL)
        
        # find docu
        for document_URL in documents_URL:
            download_pdf = final_PDF_URL(document_URL, 'data/') #download and get pdf info
            document_json.append(download_pdf) # save doc info to json        

No 'strong' in this line
No 'strong' in this line
saving -- S. M. Africanus_1849-09-13.pdf
No 'strong' in this line
No 'strong' in this line
saving -- Solomon R. Alexander_1840-08-03.pdf
not a PDF
No 'strong' in this line
No 'strong' in this line
saving -- William G. Allen_1853-12-01.pdf
No 'strong' in this line
No 'strong' in this line
saving -- William G. Allen_1853.pdf
No 'strong' in this line
No 'strong' in this line
saving -- William G. Allen_1855.pdf
No 'strong' in this line
No 'strong' in this line
saving -- William G. Allen_1852-06-22.pdf
No 'strong' in this line
No 'strong' in this line
saving -- William G. Allen_1853_694.pdf
No 'strong' in this line
No 'strong' in this line
saving -- William G. Allen_1855_372.pdf
No 'strong' in this line
No 'strong' in this line
saving -- William G. Allen_1854.pdf
No 'strong' in this line
No 'strong' in this line
saving -- William Johnson Alston_1861-09-26.pdf
No 'strong' in this line
No 'strong' in this line
saving -- John Anderson_1863.pdf


not a PDF
not a PDF
not a PDF
not a PDF
not a PDF
not a PDF
not a PDF
not a PDF
not a PDF
not a PDF
not a PDF
not a PDF
not a PDF
No 'strong' in this line
No 'strong' in this line
saving -- Octavius V. Catto_1864-05-10.pdf
No 'strong' in this line
No 'strong' in this line
saving -- William T. Catto_1859-10-24.pdf
No 'strong' in this line
No 'strong' in this line
saving -- William Thomas Catto_1862-08-25.pdf
No 'strong' in this line
No 'strong' in this line
saving -- Edward V. Clark_1850-10-01.pdf
No 'strong' in this line
No 'strong' in this line
saving -- Peter H. Clark_1859.pdf
No 'strong' in this line
No 'strong' in this line
saving -- Peter H. Clark_1858.pdf
No 'strong' in this line
No 'strong' in this line
saving -- Peter Humphries Clark_1853-08-08.pdf
No 'strong' in this line
No 'strong' in this line
saving -- Lewis Clarke_1843.pdf
No 'strong' in this line
No 'strong' in this line
saving -- Thomas Cole_1838.pdf
No 'strong' in this line
No 'strong' in this line
saving -- Thomas Col

No 'strong' in this line
No 'strong' in this line
saving -- H. Ford Douglass_1851-01-15.pdf
No 'strong' in this line
No 'strong' in this line
saving -- H. Ford Douglass_1861.pdf
No 'strong' in this line
No 'strong' in this line
saving -- H. Ford Douglass_1856-11-15.pdf
No 'strong' in this line
No 'strong' in this line
saving -- Hezekiah Ford Douglass_1850-08-01.pdf
No 'strong' in this line
No 'strong' in this line
saving -- Sarah M. Douglass_1859.pdf
No 'strong' in this line
No 'strong' in this line


KeyError: 'Date published'